### Persistance

In [51]:
from dotenv import load_dotenv

load_dotenv()

True

In [52]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults

In [53]:
tool = TavilySearchResults(max_results=2)

In [54]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

In [55]:
# %pip install langgraph-checkpoint-sqlite

In [56]:
# adding persistence
# the persistence is added by introducing checkpoints to the agent
# to achieve this SqliteSaver is used which is an in-memory database (persistent)

from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [57]:
class Agent:
    def __init__(self, model, tools, checkpointer, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_llm)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges(
            "llm",
            self.exists_action,
            {True: 'action', False: END}
        )
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile(checkpointer=checkpointer)
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def call_llm(self, state:AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}
    
    def take_action(self, state:AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for tool in tool_calls:
            print(f"Calling: {tool}")
            if not tool['name'] in self.tools:
                print("\n ...bad tool name....")
                result = "bad tool name, retry"
            else:
                result = self.tools[tool['name']].invoke(tool['args'])
            results.append(ToolMessage(tool_call_id=tool['id'], name=tool['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}


In [58]:
import os

prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""

# using groq api
open_ai_key = os.getenv('OPEN_API_KEY')
open_ai_base_url = "https://api.groq.com/openai/v1"
model = ChatOpenAI(model = 'llama3-8b-8192', api_key=open_ai_key, base_url=open_ai_base_url)

# initiate the agent
abot = Agent(model, [tool], checkpointer=memory, system=prompt)

### Streaming

In [ ]:
# streaming the messages

messages = [HumanMessage(content="Who won the the Champions Trophy 2025, and how many times did it won before?")]

# adding thread config, to keep track of different threads
thread = {"configurable": {"thread_id": "1"}}

# notice that we now invoke the agent using stream method instead of invoke method
# we get the stream of api calls and the results

for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v['messages'])

[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_d936', 'function': {'arguments': '{"query":"Champions Trophy 2025 winner"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 76, 'prompt_tokens': 1026, 'total_tokens': 1102, 'completion_tokens_details': None, 'prompt_tokens_details': None, 'queue_time': 0.021050634999999984, 'prompt_time': 0.135510174, 'completion_time': 0.063333333, 'total_time': 0.198843507}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_179b0f92c9', 'id': 'chatcmpl-84c794cb-c407-4eb5-a396-61f4c1b2f610', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-835fe349-2aae-4316-8cca-9a7c47f784c5-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'Champions Trophy 2025 winner'}, 'id': 'call_d936', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1026, 'output_tokens': 76, 'total_tokens': 1102, 'input_token_details': {}, 'o

### streamming tokens

In [62]:
memory = MemorySaver()
abot = Agent(model, [tool], checkpointer=memory, system=prompt)

In [ ]:
messages = [HumanMessage(content="What is the weather in ?")]
# adding different thread
thread = {"configurable": {"thread_id": "2"}}

async for event in abot.graph.astream_events({"messages": messages}, thread):
    kind = event["event"]
    if kind == "on_chain_stream":
        content = event['data']['chunk']['messages'][0].content
        if content:
            print(content)

What is the weather in TX?
I'm not looking at the weather in TX right now.


KeyError: 'messages'